In [3]:
import pandas as pd
import numpy as np

In [12]:
import matplotlib.pyplot as plt
from numpy import array
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [13]:
### DESCRIPTION : This function creates a Sequential (LSTM) model
### INPUT : 
###    1. Input Array, Output Array, 
###    2. Number of time steps to be used for creating sequential model
### OUPUT : 
###    1. LSTM Model
def get_model_LSTM(p_x_arr,p_y_arr,p_steps):
    X = p_x_arr.reshape((p_x_arr.shape[0], p_x_arr.shape[1], 1))
    
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(p_steps, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, p_y_arr, epochs=200, verbose=0)
    
    return model

In [14]:
### DESCRIPTION : This function creates features and labels for a sequential model
### INPUT : 
###    1. Input series in the form of dataframe row 
###    2. Number of time steps to be used for creating features and labels for a sequential model
### OUPUT : 
###    1. Feature Array for sequential model
###    2. Target Array for sequential model

def get_seq(p_row,n_steps):
    
    min_nonzero_idx = np.min(np.nonzero(p_row.tolist()),initial=0)
    max_nonzero_idx = np.max(np.nonzero(p_row.tolist()),initial=len(p_row))
    
    nonzero_len = max_nonzero_idx - min_nonzero_idx
    
    X, y = list(), list()
    for i in range(len(p_row)):
        end_ix = i + n_steps
        if end_ix > len(p_row)-1:
            break
        seq_x, seq_y = p_row[i:end_ix], p_row[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [25]:
### DESCRIPTION : This function predicts the next time step
### INPUT : 
###    1. Sequential Model
###    2. Input Array on which prediction needs to be run
###    3. Number of time steps to be used for predicting next time step

### OUPUT : 
###    1. Next time step prediction

def get_forecast(p_model,p_arr,n_steps):
    x_input = p_arr
    x_input = x_input.reshape((1, n_steps, 1))
    yhat = p_model.predict(x_input, verbose=0)
    
    return yhat

In [49]:
### DESCRIPTION : This is a wrapper function which generates features & labels , builds model and predicts next time steps 
### INPUT : 
###    1. Input data.
###    2. Time steps to be used for generating data, build model and predict next time steps.
###    3. Number of time steps for which prediction needs to be generated.

### OUPUT : 
###    1. Next time step prediction

def get_pred_meta(p_row,n_steps,p_hrzn_cnts):

    X, y = get_seq(p_row,n_steps)
    
    v_mod = get_model_LSTM(X, y, n_steps)
    
    v_pred_start_idx, v_pred_end_idx = (len(p_row)-n_steps), len(p_row)
    v_pred_inp_ini_arr = array(p_row[v_pred_start_idx:v_pred_end_idx])
    
    for ele in range(0,p_hrzn_cnts):

        if ele == 0: v_pred_inp_new_arr = v_pred_inp_ini_arr

        v_nxt_pred = get_forecast(v_mod,v_pred_inp_new_arr,n_steps)
        v_nxt_pred = (np.rint(v_nxt_pred)).astype(int)
        if ele == 0:
            v_cons_pred = v_nxt_pred
        else:
            v_cons_pred = np.append(v_cons_pred,v_nxt_pred)
        v_pred_inp_new_arr = np.append(v_pred_inp_new_arr,v_nxt_pred)
        v_pred_inp_new_arr = v_pred_inp_new_arr[len(v_pred_inp_new_arr)-n_steps:]
        print("***"*20)
    
    print(p_row)
    print(pd.Series(v_cons_pred),type(v_cons_pred))
    return "Success"

In [53]:
lst1 = [0,0,1,2,99,99,99,7]
lst2 = [0,1,2,3,99,89,89,0]
lst3 = [0,2,3,4,99,63,63,3]
lst4 = [0,3,4,5,99,43,43,0]

lst5 = [0,4,5,6,99,67,67,0]
lst6 = [0,5,6,7,99,73,73,0]
lst7 = [0,6,7,8,99,57,57,8]
lst8 = [0,7,8,9,99,64,64,0]

lst9 = [0,8,9,10,99,33,33,0]
lst10 = [0,9,10,11,99,15,15,5]
lst11 = [0,10,11,12,99,7,12,0]
lst12 = [0,11,12,13,99,2,9,0]

dummy_data = pd.DataFrame(zip(lst1,lst2,lst3,lst4,lst5,lst6,lst7,lst8,lst9,lst10,lst11,lst12),columns=['Jan2020','Feb2020','Mar2020','Apr2020','May2020','Jun2020','Jul2020','Aug2020','Sep2020','Oct2020','Nov2020','Dec2020'])

In [54]:
dummy_data

,Jan2020,Feb2020,Mar2020,Apr2020,May2020,Jun2020,Jul2020,Aug2020,Sep2020,Oct2020,Nov2020,Dec2020
0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,2,3,4,5,6,7,8,9,10,11
2,1,2,3,4,5,6,7,8,9,10,11,12
3,2,3,4,5,6,7,8,9,10,11,12,13
4,99,99,99,99,99,99,99,99,99,99,99,99
5,99,89,63,43,67,73,57,64,33,15,7,2
6,99,89,63,43,67,73,57,64,33,15,12,9
7,7,0,3,0,0,0,8,0,0,5,0,0


In [56]:
dummy_data.apply(get_pred_meta, n_steps=6,p_hrzn_cnts = 6,axis=1)

************************************************************
************************************************************
************************************************************
************************************************************
************************************************************
************************************************************
Jan2020    0
Feb2020    0
Mar2020    0
Apr2020    0
May2020    0
Jun2020    0
Jul2020    0
Aug2020    0
Sep2020    0
Oct2020    0
Nov2020    0
Dec2020    0
Name: 0, dtype: int64
0    0
1    0
2    0
3    0
4    0
5    0
dtype: int64 <class 'numpy.ndarray'>
************************************************************
************************************************************
************************************************************
************************************************************
************************************************************
************************************************************
Jan2020    

0    Success
1    Success
2    Success
3    Success
4    Success
5    Success
6    Success
7    Success
dtype: object